# Semillero

### Autor: Federico Picado
### Fecha de última modificación: 28/10/2024
### Descripción: Entrenar y guardar modelos solamente cambiando la semillla.

Este script creara N modelos cambiando solo la semilla y se guardaran en la carpeta especificada, de la siguiente forma :

- `<modelo_nombre>_<numero_seed>.ipynb`

## Parámetro

N_Seeds: Cantidad de semillas aleatoria de 1000 a 100000 elegir.

In [1]:
N_Seeds= 4
meses_train = [202012,202101,202102,202103,202104]
mes_test = 202106

## Input

In [2]:
# Ya en el input se ejecutan cosas
import pandas as pd
import numpy as np
import random
import lightgbm as lgb
import optuna
import os

/home/fedepicado/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/fedepicado/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/modelos/'
db_path = base_path + 'buckets/b1/db/'
dataset_path = base_path + 'buckets/b1/datasets/'
exp_path = base_path + 'buckets/b1/exp/'
dataset_file = 'competencia_02.parquet'
full_path = dataset_path + dataset_file

# carpeta donde se guardan los modelos
nombre_carpeta = "modelo_base_1" # Cambiar
modelo_nombre= "modelo_base_1"# Cambiar

####################################################
db_file_name = "optimizationes_base.db"
db_file_path = os.path.join(db_path, db_file_name)
# me traigo la optimizacion del modelo
storage_name = f"sqlite:///{db_file_path}"  
study_name = "modelo_base_1"

In [4]:
# creo el study
study_1= optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

study=study_1
best_iter=study_1.best_trial.user_attrs["best_iter"]

# Definir los mejores parámetros del modelo
best_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'verbose': 0
    #agregue los que su modelo considere
}

[I 2024-11-17 14:21:29,011] Using an existing study with name 'modelo_base' instead of creating a new one.


Esta parte podria estar en procesamiento pero creo que aca se va a poder modificar mejor en el caso que el modelo no sea el mismo

In [5]:
### El modelo que entre yo es un LGBM, creando la clase binaria y separe los datos para entrenamiento
data = pd.read_parquet(dataset_path + dataset_file)
df=data.copy()

In [6]:
df['clase_peso'] = 1.0
df['clase_binaria'] = 0

df.loc[df['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.0002
df.loc[df['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.0001

df['clase_binaria'] = np.where(df['clase_ternaria'] == 'BAJA+2', 1, 0)

### train. Los datos con los que realize la optimizacion
train_data = df[df['foto_mes'].isin(meses_train)]
X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = train_data['clase_binaria']
w_train = train_data['clase_peso']

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [7]:
# donde se va a crear la carpeta con los modelos
modelos_path = exp_path + 'modelos/'
# dentro de modelos_path se creara una carpeta y se guardaran los modelos entrenados

# MAIN

### Funciones 

In [8]:
def entrenar_modelos_con_varias_semillas(best_params, X_train, y_train_binaria, w_train, best_iter, semillas,modelo_nombre):
    """
    Entrena modelos de LightGBM utilizando los mejores parámetros y una lista de semillas.
    
    Parámetros:
    - best_params: dict, contiene los mejores parámetros del estudio de optimización.
    - X_train: dataframe, las características de entrenamiento.
    - y_train_binaria: array-like, la variable objetivo binaria.
    - w_train: array-like, los pesos para las observaciones de entrenamiento.
    - best_iter: int, la mejor cantidad de árboles según el estudio de optimización.
    - semillas: lista de int, las semillas para entrenar cada modelo.
    
    Retorna:
    - modelos_entrenados: diccionario de modelos de LightGBM entrenados, con nombres basados en la semilla.
    """
    modelos_entrenados = {}

    # Crear el dataset de entrenamiento
    train_data = lgb.Dataset(X_train, label=y_train_binaria, weight=w_train)

    # Entrenar un modelo por cada semilla
    for seed in semillas:
        # Actualizar el parámetro de la semilla
        params = best_params.copy()
        params['seed'] = seed
        
        # Entrenar el modelo
        model = lgb.train(params, train_data, num_boost_round=best_iter)
        
        # Asignar el nombre del modelo como "modelo_semilla"
        nombre_modelo = f"{modelo_nombre}_{seed}"
        modelos_entrenados[nombre_modelo] = model
        
        print(f"Modelo entrenado con semilla {seed} guardado como {nombre_modelo}")
    
    return modelos_entrenados


def crear_semillas_primas_aleatorias(N, inicio=1000, fin=100000):
    def es_primo(n):
        if n < 2:
            return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False
        return True

    semillas = set()  # Usamos un conjunto para evitar duplicados
    while len(semillas) < N:
        num = random.randint(inicio, fin)
        if es_primo(num):
            semillas.add(num)
    return list(semillas)

In [9]:
# Creo las semillas
semillas = crear_semillas_primas_aleatorias(N_Seeds)

In [10]:
# Entrenar los modelos
modelos_entrenado = entrenar_modelos_con_varias_semillas(best_params, X_train, y_train_binaria, w_train, best_iter, semillas, modelo_nombre)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [11]:
def crear_carpeta(base_ruta, nombre_carpeta):
    """
    Crea una carpeta en la ruta especificada si no existe y devuelve la ruta completa.
    
    Parámetros:
    - base_ruta: str, ruta base donde se creará la carpeta.
    - nombre_carpeta: str, nombre de la carpeta a crear.
    
    Retorna:
    - ruta_completa: str, la ruta completa de la carpeta creada.
    """
    # Construye la ruta completa de la carpeta
    ruta_completa = os.path.join(base_ruta, nombre_carpeta)
    
    # Verifica si la carpeta ya existe; si no, la crea
    if not os.path.exists(ruta_completa):
        os.makedirs(ruta_completa)
        print(f"Carpeta creada en: {ruta_completa}")
    else:
        print(f"La carpeta ya existe en: {ruta_completa}")
    
    # Retorna la ruta completa de la carpeta
    return ruta_completa

In [12]:
ruta_modelos=crear_carpeta(exp_path, nombre_carpeta)

Carpeta creada en: /home/fedepicado/buckets/b1/exp/modelo_base


In [13]:
def guardar_modelo(model, modelos_carpeta_path, nombre_archivo):
    """
    Guarda un modelo de LightGBM en un archivo especificado.
    
    Parámetros:
    - model: el modelo de LightGBM entrenado.
    - modelos_path: str, la ruta de la carpeta donde se guardará el modelo.
    - nombre_archivo: str, el nombre del archivo (incluyendo la extensión .txt) para guardar el modelo.
    """
    # Asegurar que la carpeta existe
    if not os.path.exists(modelos_carpeta_path):
        os.makedirs(modelos_carpeta_path)
    
    # Construir la ruta completa del archivo
    ruta_completa = os.path.join(modelos_carpeta_path, nombre_archivo)
    
    # Guardar el modelo en la ruta especificada
    model.save_model(ruta_completa)

In [14]:
for nombre_modelo, modelo in modelos_entrenado.items():
    guardar_modelo(modelo, ruta_modelos, f"{nombre_modelo}.txt")